In [1]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [2]:
config = {
    'user': 'root',
    'password': '>D"@p5dgh}@K)C6)',
    'host': 'localhost',
    'database': 'data',
    #'auth_plugin' : 'mysql_native_password'
}

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()

In [3]:
def markdown_table(res, titles):
    if len(titles) != len(res[0]):
        raise Exception("arg arrays don't match")

    table = ' | '.join(titles)
    table += '\n'
    table += ' | '.join([":---:"] * len(titles))
    table += '\n'

    for row in res:
        table += ' | '.join([str(elem) for elem in row])
        table += '\n'

    return table

In [147]:
###QUERY 3 - WORKS
cursor.execute("""SELECT
  make.description,
  COUNT(*)
FROM
  Vehicles v,
  Victims vict,
  Parties p,
  VehiculeMake make,
  VictimDegreeOfInjury injury
WHERE
  v.party_id = p.id
  AND vict.case_id = p.case_id
  AND v.vehicle_make = make.id
  AND vict.victim_degree_of_injury = injury.id
  AND (
    injury.description = 'Killed'
    OR injury.description = 'Severe Injury'
  )
GROUP BY
  v.vehicle_make
ORDER BY
  COUNT(*) DESC
LIMIT 10
""")

In [148]:
res = cursor.fetchall()

In [149]:
res

[('FORD', 31727),
 ('CHEVROLET', 22835),
 ('TOYOTA', 22292),
 ('HONDA', 20212),
 ('DODGE', 9028),
 ('NISSAN', 8070),
 ('GMC', 4696),
 ('NOT STATED', 4172),
 ('HARLEY-DAVIDSON', 3810),
 ('MISCELLANEOUS', 3753)]

In [90]:
### QUERY 4 - DOESNT WORK
cursor.execute("""SELECT
  v.victim_seating_position,
  COUNT(*) / (
    SELECT
      COUNT(*)
    FROM
      Victims
    WHERE 
    v.victim_seating_position IS NOT NULL
  )
FROM
  Victims v,
  VictimDegreeOfInjury i
WHERE
  v.victim_degree_of_injury = i.id
  AND i.description = 'No Injury' AND
  v.victim_seating_position IS NOT NULL
GROUP BY
  (v.victim_seating_position)
""")

In [91]:
res = cursor.fetchall()

In [92]:
print(markdown_table(res,['Seating','Safety %']))

Seating | Safety %
:---: | :---:
3 | 0.2360
2 | 0.0112
6 | 0.0879
4 | 0.0717
9 | 0.0173
0 | 0.0110
8 | 0.0083
5 | 0.0335
7 | 0.0104
1 | 0.0029



In [102]:
#QUERY 5 - WORKS ??
cursor.execute("""
SELECT COUNT(*)
FROM (SELECT statewide_vehicle_type  ,COUNT(Parties.case_id) as collisions ,COUNT(DISTINCT county_city_location) as city
      FROM (Vehicles   INNER JOIN Parties ON Parties.id = Vehicles.party_id 
                       INNER JOIN Locations ON Locations.case_id = Parties.id) 
GROUP BY statewide_vehicle_type 
) A
WHERE A.collisions > 10 AND
      A.city > (
SELECT COUNT(DISTINCT county_city_location)
FROM Locations)/2
""")

In [103]:
res = cursor.fetchall()

In [104]:
res

[(15,)]

In [59]:
### QUERY 6 - WORKS

cursor.execute("""(SELECT
  cities.location_id,
  l.population,
  l.case_id,
  AVG(v.victim_age) as victim_mean
FROM
  (SELECT
  l.county_city_location as location_id
FROM
  Locations l
WHERE
  l.population != 9
  AND l.population != 0
GROUP BY
  l.county_city_location,
  l.population
ORDER BY
  l.population DESC
LIMIT
  1 OFFSET 0) cities,
  Locations l,
  Victims v
WHERE
  (l.county_city_location = cities.location_id)
  AND v.case_id = l.case_id
GROUP BY
  v.case_id,
  l.case_id,
  l.county_city_location,
  l.population
ORDER BY
  case when AVG(v.victim_age) is null then 1 when AVG(v.victim_age) then 1 else 0 end,
  AVG(v.victim_age)
LIMIT
  10) UNION 
  
  (SELECT
  cities.location_id,
  l.population,
  l.case_id,
  AVG(v.victim_age) as victim_mean
FROM
  (SELECT
  l.location_id as location_id
FROM
  (
    SELECT
      l.county_city_location as location_id,
      l.population as pop
    FROM
      Locations l
    WHERE
      l.population != 9
      AND l.population != 0
    GROUP BY
      l.county_city_location,
      l.population
    ORDER BY
      l.population DESC
    LIMIT
      2 OFFSET 1
  ) l
ORDER BY
  l.pop ASC
LIMIT
  1) cities,
  Locations l,
  Victims v
WHERE
  (l.county_city_location = cities.location_id)
  AND v.case_id = l.case_id
GROUP BY
  v.case_id,
  l.case_id,
  l.county_city_location,
  l.population
ORDER BY
  case when AVG(v.victim_age) is null then 1 when AVG(v.victim_age) then 1 else 0 end,
  AVG(v.victim_age)
LIMIT
  10) UNION 
  
  
  (SELECT
  cities.location_id,
  l.population,
  l.case_id,
  AVG(v.victim_age) as victim_mean
FROM
  (SELECT
  l.county_city_location as location_id
FROM
  Locations l
WHERE
  l.population != 9
  AND l.population != 0
GROUP BY
  l.county_city_location,
  l.population
ORDER BY
  l.population DESC
LIMIT
  1 OFFSET 3) cities,
  Locations l,
  Victims v
WHERE
  (l.county_city_location = cities.location_id)
  AND v.case_id = l.case_id
GROUP BY
  v.case_id,
  l.case_id,
  l.county_city_location,
  l.population
ORDER BY
  case when AVG(v.victim_age) is null then 1 when AVG(v.victim_age) then 1 else 0 end,
  AVG(v.victim_age)
LIMIT
  10)  """)

In [60]:
res = cursor.fetchall()

In [61]:
res

[(3711, 7, '0060439', Decimal('0.0000')),
 (3711, 7, '0162669', Decimal('0.0000')),
 (3711, 7, '0382996', Decimal('0.0000')),
 (3711, 7, '0347879', Decimal('0.0000')),
 (3711, 7, '0695315', Decimal('0.0000')),
 (3711, 7, '0568761', Decimal('0.0000')),
 (3711, 7, '0186147', Decimal('0.0000')),
 (3711, 7, '1034588', Decimal('0.0000')),
 (3711, 7, '0197188', Decimal('0.0000')),
 (3711, 7, '0066852', Decimal('0.0000')),
 (1005, 7, '2048203', Decimal('0.0000')),
 (1005, 7, '2376747', Decimal('0.0000')),
 (1005, 7, '0682784', Decimal('0.0000')),
 (1005, 7, '1837508', Decimal('0.0000')),
 (1005, 7, '2399236', Decimal('0.0000')),
 (1005, 7, '0644343', Decimal('0.0000')),
 (1005, 7, '0800448', Decimal('0.0000')),
 (1005, 7, '0457868', Decimal('0.0000')),
 (1005, 7, '0360320', Decimal('0.0000')),
 (1005, 7, '1186635', Decimal('0.0000')),
 (3019, 7, '2072101', Decimal('0.0000')),
 (3019, 7, '2674015', Decimal('0.0000')),
 (3019, 7, '2138547', Decimal('0.0000')),
 (3019, 7, '1170908', Decimal('0.0

In [57]:
### QUERY 7 - WORKS
cursor.execute("""SELECT
  old_people_collisions.id,
  old_people_collisions.max_age
FROM(
    SELECT
      v.case_id AS id,
      MAX(v.victim_age) AS max_age
    FROM
      Victims v
    GROUP BY
      v.case_id
    HAVING
      MIN(v.victim_age) > 99
  ) AS old_people_collisions,
  Collisions c,
  TypeOfCollision tc
WHERE(
    c.case_id = old_people_collisions.id
    AND c.type_of_collision = tc.id
    AND tc.description = 'pedestrian'
  )""")

In [58]:
cursor.fetchall()

[('0036446', 110),
 ('0069198', 101),
 ('0415838', 100),
 ('0439197', 102),
 ('0445265', 101),
 ('0486529', 100),
 ('0566220', 102),
 ('0621752', 100),
 ('0644226', 103),
 ('0784061', 102),
 ('0817210', 102),
 ('0820619', 101),
 ('0828116', 102),
 ('0851026', 106),
 ('0868472', 103),
 ('0885420', 100),
 ('1209166', 101),
 ('1213340', 121),
 ('1347636', 101),
 ('1373664', 101),
 ('1548445', 102),
 ('1847678', 104),
 ('2290129', 100),
 ('2427260', 100),
 ('2472739', 103),
 ('2531557', 103),
 ('3388544', 105),
 ('3485436', 101)]

In [59]:
res

[(1942, 399582),
 (1900, 118446),
 (3400, 80191),
 (3711, 76867),
 (109, 72995),
 (3300, 61453),
 (3404, 58068),
 (4313, 57852),
 (1941, 53565),
 (3801, 48450)]

In [51]:
###QUERY 8 - WORKS
cursor.execute("""SELECT
  makes.description,
  v.vehicle_year,
  type.description,
  COUNT(*) AS num_of_collisions
FROM
  Vehicles v,
  Parties p,
  Collisions c,
  VehiculeMake makes,
  StatewideVehiculeType type
WHERE
  v.party_id = p.id AND v.vehicle_make != 'None'
  AND p.case_id = c.case_id AND
  v.vehicle_make = makes.id AND
  v.statewide_vehicle_type = type.id
GROUP BY v.vehicle_make,
  v.vehicle_year,
  v.statewide_vehicle_type
HAVING(COUNT(*) > 9)
ORDER BY
  num_of_collisions DESC
""")

In [52]:
res = cursor.fetchall()

In [53]:
res

[('TOYOTA', 2000, 'passenger car', 52504),
 ('FORD', 2000, 'passenger car', 51943),
 ('HONDA', 2000, 'passenger car', 50284),
 ('FORD', 1998, 'passenger car', 49182),
 ('TOYOTA', 2001, 'passenger car', 47232),
 ('HONDA', 2001, 'passenger car', 45277),
 ('FORD', 2001, 'passenger car', 45236),
 ('TOYOTA', 1999, 'passenger car', 42941),
 ('HONDA', 1998, 'passenger car', 42091),
 ('FORD', 1999, 'passenger car', 41948),
 ('FORD', 1995, 'passenger car', 40246),
 ('HONDA', 1997, 'passenger car', 39210),
 ('FORD', 1997, 'passenger car', 38885),
 ('HONDA', 1999, 'passenger car', 38556),
 ('TOYOTA', 2002, 'passenger car', 38427),
 ('TOYOTA', 1998, 'passenger car', 38012),
 ('TOYOTA', 1997, 'passenger car', 37158),
 ('TOYOTA', 2003, 'passenger car', 35943),
 ('HONDA', 2002, 'passenger car', 35785),
 ('FORD', 2002, 'passenger car', 35460),
 ('FORD', 1996, 'passenger car', 35307),
 ('HONDA', 1995, 'passenger car', 34657),
 ('TOYOTA', 1996, 'passenger car', 33697),
 ('FORD', 1993, 'passenger car', 3

In [54]:
### QUERY 9 - WORKS
cursor.execute("""SELECT
  l.county_city_location,
  COUNT(*)
FROM
  Collisions c,
  Locations l
WHERE
  (c.case_id = l.case_id)
GROUP BY
  (l.county_city_location)
ORDER BY
  COUNT(*) DESC
LIMIT
  10""")

In [55]:
res = cursor.fetchall()

In [56]:
res

[(1942, 399582),
 (1900, 118446),
 (3400, 80191),
 (3711, 76867),
 (109, 72995),
 (3300, 61453),
 (3404, 58068),
 (4313, 57852),
 (1941, 53565),
 (3801, 48450)]

In [37]:
### QUERY 10 - WORKS
cursor.execute("""SELECT CASE WHEN l.description = 'daylight'
  OR (
    f.lighting = NULL
    AND c.collision_time BETWEEN '00:08:00'
    AND '17:59:00'
  ) THEN 'day' WHEN (
    f.lighting = NULL
    AND c.collision_time BETWEEN '00:06:00'
    AND '07:59:00'
  )
  OR (
    l.description = 'dusk or dawn'
    AND (
      c.collision_time BETWEEN '00:06:00'
      AND '07:59:00'
    )
  ) THEN 'dawn' WHEN (
    f.lighting = NULL
    AND c.collision_time BETWEEN '00:18:00'
    AND '19:59:00'
  )
  OR (
    l.description = 'dusk or dawn'
    AND (
      c.collision_time BETWEEN '00:18:00'
      AND '19:59:00'
    )
  ) THEN 'dusk' ELSE 'night' END AS time_of_day,
  count(1) as num
FROM
  Factors f,
  Lighting l,
  Collisions c
WHERE
  f.case_id = c.case_id
  AND (
    f.lighting = l.id
    OR f.lighting = NULL
  )
GROUP BY
  time_of_day""")

In [38]:
res = cursor.fetchall()

In [39]:
res

[('night', 1039682), ('day', 2496938), ('dusk', 74906), ('dawn', 40756)]